In [20]:
from dotenv import load_dotenv
from IPython.display import display, HTML, JSON, Markdown
load_dotenv()

True

In [21]:
from langchain.tools import tool, ToolRuntime

@tool
def read_email(runtime: ToolRuntime) -> str:
    """Read an email from the given address."""
    # take email from state
    return runtime.state["email"]

@tool
def send_email(body: str) -> str:
    """Send an email to the given address with the given subject and body."""
    # fake email sending
    return f"Email sent"

In [22]:
from langchain.agents import create_agent
from langchain_aws import ChatBedrock

# 1. CONFIGURACIÓN PARA DEEPSEEK-R1 (Razonamiento Complejo)
# Ideal para agentes que necesitan planificar pasos lógicos.
# llm = ChatBedrock(
#     model_id="us.deepseek.r1-v1:0",  # ID oficial validado
#     region_name="us-east-1",
#     model_kwargs={
#         "temperature": 0.6, # DeepSeek recomienda 0.6 para razonamiento
#         "max_tokens": 8192,  # Recomendado para no degradar calidad del CoT
#         "top_p": 0.95,
#     }
# )


# llm = ChatBedrock(
#     model_id="us.deepseek.v3-v1:0", # Prueba este primero
#     region_name="us-east-1",        # O us-west-2
#     model_kwargs={
#         "temperature": 0.7,
#         "max_tokens": 4096
#     }
# )
# from langchain_aws import ChatBedrock
# llm = ChatBedrock(
# model_id="us.meta.llama4-scout-17b-instruct-v1:0",  # Nota el prefijo "us."
# # model_id="cohere.command-r-plus-v1:0",
# region_name="us-east-1",
# model_kwargs={
# "temperature": 0.5,
# "max_tokens": 2048,
# "top_p": 0.9,
# }
# )


# llm = ChatBedrock(
#     model_id="us.meta.llama4-maverick-17b-instruct-v1:0",  # Nota el prefijo "us."
#     region_name="us-east-1",
#     model_kwargs={
#         "temperature": 0.5,
#         "max_tokens": 2048,
#         "top_p": 0.9,
#     }
# )

llm = ChatBedrock(
    model_id="us.meta.llama4-scout-17b-instruct-v1:0",  # Nota el prefijo "us."
    region_name="us-east-1",
    model_kwargs={
        "temperature": 0.5,
        "max_tokens": 2048,
        "top_p": 0.9,
    }
)


In [23]:
from langchain.agents import create_agent, AgentState
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware

class EmailState(AgentState):
    email: str

agent = create_agent(
    model=llm,
    system_prompt="""You are an email assistant. You MUST use the available tools to complete tasks.
CRITICAL RULES:
1. To read emails, you MUST call the read_email tool
2. To send emails, you MUST call the send_email tool  
3. NEVER respond without using a tool first
4. Do NOT explain what you would do - just DO IT by calling the tool
Available tools:
- read_email: Use this to read the user's email
- send_email: Use this to send an email response
START BY CALLING A TOOL.""",
    tools=[read_email, send_email],
    state_schema=EmailState,
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "read_email": False,
                "send_email": True,
            },
            description_prefix="Tool execution requires approval",
        ),
    ],
)

In [24]:
# llm = ChatBedrock(
#     model_id="us.meta.llama4-maverick-17b-instruct-v1:0",
#     region_name="us-east-1",
#     model_kwargs={"temperature": 0.1, "max_tokens": 2048}
# )

# llm_with_tools = llm.bind_tools([read_email, send_email], tool_choice="required")

In [25]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {
        "messages": [HumanMessage(content="Please read my email and send a response.")],
        "email": "Hi Seán, I'm going to be late for our meeting tomorrow. Can we reschedule? Best, John."
    },
    config=config
)

In [26]:
from pprint import pprint
pprint(response)

# O más específico:
print("Keys:", response.keys())
print("Messages:", len(response.get('messages', [])))

# Ver el último mensaje del LLM
if 'messages' in response:
    last_msg = response['messages'][-1]
    print("Último mensaje:", last_msg.content[:200] if hasattr(last_msg, 'content') else last_msg)
    print("Tool calls:", getattr(last_msg, 'tool_calls', None))

{'__interrupt__': [Interrupt(value={'action_requests': [{'args': {'body': 'Hi '
                                                                          'John, '
                                                                          'No '
                                                                          'worries, '
                                                                          'we '
                                                                          'can '
                                                                          'reschedule '
                                                                          'the '
                                                                          'meeting. '
                                                                          'Let '
                                                                          'me '
                                                                          'know '
            

In [27]:
from pprint import pprint

pprint(response)

{'__interrupt__': [Interrupt(value={'action_requests': [{'args': {'body': 'Hi '
                                                                          'John, '
                                                                          'No '
                                                                          'worries, '
                                                                          'we '
                                                                          'can '
                                                                          'reschedule '
                                                                          'the '
                                                                          'meeting. '
                                                                          'Let '
                                                                          'me '
                                                                          'know '
            

In [29]:
print(response['__interrupt__'])

[Interrupt(value={'action_requests': [{'name': 'send_email', 'args': {'body': 'Hi John, No worries, we can reschedule the meeting. Let me know a time that works better for you. Best, Seán'}, 'description': "Tool execution requires approval\n\nTool: send_email\nArgs: {'body': 'Hi John, No worries, we can reschedule the meeting. Let me know a time that works better for you. Best, Seán'}"}], 'review_configs': [{'action_name': 'send_email', 'allowed_decisions': ['approve', 'edit', 'reject']}]}, id='2167fc22d57042091870ddabbea7a50b')]


In [10]:
# Access just the 'body' argument from the tool call
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

Hi John,

I understand. Let's reschedule the meeting to a time that works better for you. Please let me know your availability.

Best,
Seán


## Approve

In [11]:
from langgraph.types import Command

response = agent.invoke(
    Command( 
        resume={"decisions": [{"type": "approve"}]}
    ), 
    config=config # Same thread ID to resume the paused conversation
)

pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response.', additional_kwargs={}, response_metadata={}, id='0244889e-9cce-4f62-9d57-45e6db677789'),
              AIMessage(content=[{'type': 'text', 'text': "<thinking>I need to read the email first to understand its content and then send a response. I will start by calling the read_email tool to read the user's email.</thinking>\n"}, {'type': 'tool_use', 'name': 'read_email', 'input': {}, 'id': 'tooluse_ZIDP9CZDRUOdw8PB45RSJA'}], additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'd46d3e0e-7916-4b7c-979d-d1a83d5fcf59', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 03 Jan 2026 01:17:26 GMT', 'content-type': 'application/json', 'content-length': '469', 'connection': 'keep-alive', 'x-amzn-requestid': 'd46d3e0e-7916-4b7c-979d-d1a83d5fcf59'}, 'RetryAttempts': 0}, 'stopReason': 'to

## Reject

In [30]:
response = agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "reject",
                    # An explanation of why the request was rejected
                    "message": "No please sign off - Your merciful leader, Seán."
                }
            ]
        }
    ), 
    config=config # Same thread ID to resume the paused conversation
    )   

pprint(response)

{'__interrupt__': [Interrupt(value={'action_requests': [{'args': {'body': 'Hi '
                                                                          'John, '
                                                                          'No '
                                                                          'worries, '
                                                                          'we '
                                                                          'can '
                                                                          'reschedule '
                                                                          'the '
                                                                          'meeting. '
                                                                          'Let '
                                                                          'me '
                                                                          'know '
            

In [31]:
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

Hi John, No worries, we can reschedule the meeting. Let me know a time that works better for you. Best, Seán


## Edit

In [32]:
response = agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "edit",
                    # Edited action with tool name and args
                    "edited_action": {
                        # Tool name to call.
                        # Will usually be the same as the original action.
                        "name": "send_email",
                        # Arguments to pass to the tool.
                        "args": {"body": "This is the last straw, you're fired!"},
                    }
                }
            ]
        }
    ), 
    config=config # Same thread ID to resume the paused conversation
    )   

pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response.', additional_kwargs={}, response_metadata={}, id='e5875070-861a-4a61-aaf6-a0c35a958612'),
              AIMessage(content=[{'type': 'text', 'text': '<thinking>I need to read the email first to understand its content and then send an appropriate response. I will start by reading the email.</thinking>\n'}, {'type': 'tool_use', 'name': 'read_email', 'input': {}, 'id': 'tooluse_zXBQKaofSZi2omilHPRFow'}], additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'e51290d0-e928-4c1f-b859-1cdd0a3e06dc', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 03 Jan 2026 01:21:38 GMT', 'content-type': 'application/json', 'content-length': '447', 'connection': 'keep-alive', 'x-amzn-requestid': 'e51290d0-e928-4c1f-b859-1cdd0a3e06dc'}, 'RetryAttempts': 0}, 'stopReason': 'tool_use', 'metrics': {'